In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [ ]:
# Load the cleaned dataset
data = pd.read_csv("cleaned_movies_dataset.csv")

# Encoding

this part own is to split our dataset to trainset, testset and valset and do encocoding in categorical columns using OrdinalEncoder and standardisation
of numerical columns

In [4]:
# split data into training and testing sets
from sklearn.model_selection import train_test_split

target = data['US_BoxOfficeUSD']
X = data.drop(['US_BoxOfficeUSD'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
# Pipeline for categorical encoding columns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.compose import ColumnTransformer

cat_cols=data.select_dtypes(exclude=['number']).columns
target='US_BoxOfficeUSD'
num_cols=data.drop(target,axis=1).select_dtypes(include=['number']).columns

cat_pipeline=Pipeline([
    ('ordinal',OrdinalEncoder()),
    # ('scaler',RobustScaler())

])
num_pipeline=Pipeline([
    ('scaler',RobustScaler())
])
trans=ColumnTransformer([
    ('cat',cat_pipeline,cat_cols),
    ('num',num_pipeline,num_cols)
], remainder='passthrough')



In [7]:
train_data_preprocess=pd.DataFrame(trans.fit_transform(X_train),columns=[f'{col.replace("cat__","")}' if col.startswith('cat') else f'{col.replace("num__","")}' for col in trans.get_feature_names_out() ])

data_test_preprocess=pd.DataFrame(trans.transform(X_test),columns=[f'{col.replace("cat__","")}' if col.startswith('cat') else f'{col.replace("num__","")}' for col in trans.get_feature_names_out() ])

data_val_preprocess=pd.DataFrame(trans.transform(X_val),columns=[f'{col.replace("cat__","")}' if col.startswith('cat') else f'{col.replace("num__","")}' for col in trans.get_feature_names_out() ])


In [8]:
data_val_preprocess.head()

,Title,Genre,Country,Director,LeadActor,MovieID,ReleaseYear,BudgetUSD,Global_BoxOfficeUSD,Opening_Day_SalesUSD,One_Week_SalesUSD,IMDbRating,RottenTomatoesScore,NumVotesIMDb,NumVotesRT,ReleaseMonth,ReleaseDay
0,5248.0,3.0,7.0,62.0,111.0,0.734604,-1.200000,-0.335590,-0.135442,-0.339818,-0.266266,-0.15,0.458333,0.348719,-0.130311,-0.833333,-0.266667
1,6825.0,3.0,9.0,137.0,236.0,0.652838,-0.133333,0.269954,0.399211,0.645732,0.417938,0.45,1.291667,-0.610751,0.519099,-0.500000,-0.200000
2,9321.0,4.0,9.0,91.0,297.0,-0.085915,0.500000,1.002090,0.957163,1.075647,0.912283,0.75,0.875000,1.233711,0.707159,-0.166667,0.466667
3,6115.0,3.0,9.0,35.0,287.0,0.492381,-0.366667,0.415877,0.706431,0.648292,0.772801,-0.65,-0.458333,1.328757,-0.093013,-0.166667,-0.200000
4,4578.0,0.0,9.0,39.0,97.0,0.466072,0.466667,-0.749289,-0.383401,-0.677451,-0.511715,-0.25,-0.541667,0.097394,-0.193512,0.333333,-0.800000


In [9]:
data_val_preprocess.shape, y_val.shape

((160000, 17), (160000,))

# Embedding

Like the column title, director and lead actor was long text ordinal and one hot encoding are not good then we pass the ordinal output to pytorch embedding for more relation(using vectors) than exemple: 1,2,3

In [ ]:
# spliting features(Title_data, Director_data, LeadActor_data, num for embeddings) and target to tensors

# train set
train_Title_data = torch.tensor(train_data_preprocess['Title'].values,dtype=torch.long)  # titres encodés
train_Director_data  = torch.tensor(train_data_preprocess['Director'].values,dtype=torch.long)  # réalisateurs encodés
train_LeadActor_data  = torch.tensor(train_data_preprocess['LeadActor'].values,dtype=torch.long)  # acteurs principaux encodés
train_num= torch.tensor(train_data_preprocess.drop(['Title', 'Director', 'LeadActor',], axis=1).values, dtype=torch.float)  # autres features numériques
train_target=torch.tensor(y_train.values, dtype=torch.float)

# test set
test_Title_data = torch.tensor(data_test_preprocess['Title'].values,dtype=torch.long)  # titres encodés
test_Director_data  = torch.tensor(data_test_preprocess['Director'].values,dtype=torch.long)  # réalisateurs encodés
test_LeadActor_data  = torch.tensor(data_test_preprocess['LeadActor'].values,dtype=torch.long)  # acteurs principaux encodés
test_num= torch.tensor(data_test_preprocess.drop(['Title', 'Director', 'LeadActor'], axis=1).values, dtype=torch.float)  # autres features numériques
test_target=torch.tensor(y_test.values, dtype=torch.float)

# val set
val_Title_data = torch.tensor(data_val_preprocess['Title'].values,dtype=torch.long)  # titres encodés
val_Director_data  = torch.tensor(data_val_preprocess['Director'].values,dtype=torch.long)  # réalisateurs encodés
val_LeadActor_data  = torch.tensor(data_val_preprocess['LeadActor'].values,dtype=torch.long)  # acteurs principaux encodés
val_num= torch.tensor(data_val_preprocess.drop(['Title', 'Director', 'LeadActor'], axis=1).values, dtype=torch.float)  # autres features numériques
val_target=torch.tensor(y_val.values, dtype=torch.float)


In [ ]:
 # embeddings for Title_data, Director_data, LeadActor_data
emb_dim=10

# For training set
train_title_emb = nn.Embedding(num_embeddings=int(train_Title_data.max().item()) + 1, embedding_dim=emb_dim)
train_director_emb = nn.Embedding(num_embeddings=int(train_Director_data.max().item()) + 1, embedding_dim=emb_dim)
train_leadactor_emb = nn.Embedding(num_embeddings=int(train_LeadActor_data.max().item()) + 1, embedding_dim=emb_dim)

# For validation and test sets
val_title_emb = nn.Embedding(num_embeddings=int(val_Title_data.max().item()) + 1, embedding_dim=emb_dim)
val_director_emb = nn.Embedding(num_embeddings=int(val_Director_data.max().item()) + 1, embedding_dim=emb_dim)
val_leadactor_emb = nn.Embedding(num_embeddings=int(val_LeadActor_data.max().item()) + 1, embedding_dim=emb_dim)

# For test set
test_title_emb = nn.Embedding(num_embeddings=int(test_Title_data.max().item()) + 1, embedding_dim=emb_dim)
test_director_emb = nn.Embedding(num_embeddings=int(test_Director_data.max().item()) + 1, embedding_dim=emb_dim)
test_leadactor_emb = nn.Embedding(num_embeddings=int(test_LeadActor_data.max().item()) + 1, embedding_dim=emb_dim)




In [ ]:
# Getting the embeddings

# For training set
train_title_vec = train_title_emb(train_Title_data)
train_director_vec = train_director_emb(train_Director_data)
train_leadactor_vec = train_leadactor_emb(train_LeadActor_data)

# For validation and test sets
val_title_vec = val_title_emb(val_Title_data)
val_director_vec = val_director_emb(val_Director_data)
val_leadactor_vec = val_leadactor_emb(val_LeadActor_data)

# For test set
test_title_vec = test_title_emb(test_Title_data)
test_director_vec = test_director_emb(test_Director_data)
test_leadactor_vec = test_leadactor_emb(test_LeadActor_data)

# Construction and saving

we concat Title, Director, LeadActor and other columns together(emb_dim=10 for the three columns then 10*3=30 and the other 13 columns then 43 total)

In [ ]:
# Final feature set
X_train = torch.cat([train_num, train_title_vec, train_director_vec, train_leadactor_vec], dim=1)
X_val = torch.cat([val_num, val_title_vec, val_director_vec, val_leadactor_vec], dim=1)
X_test = torch.cat([test_num, test_title_vec, test_director_vec, test_leadactor_vec], dim=1)

In [ ]:
# show shapes
X_train.shape,train_target.shape

(torch.Size([999999, 43]), torch.Size([999999]))

In [13]:
X_train

tensor([[ 3.0000,  6.0000, -0.3220,  ..., -0.1171,  0.7080, -0.3780],
        [ 1.0000,  9.0000, -0.8028,  ...,  1.7534, -0.6855, -0.0862],
        [ 4.0000,  9.0000, -0.8672,  ...,  0.4404,  1.2048,  2.1722],
        ...,
        [ 0.0000,  9.0000, -0.5440,  ...,  0.4033,  1.2792, -1.2846],
        [ 6.0000,  9.0000, -0.7541,  ...,  1.6493, -0.5843, -0.1758],
        [ 1.0000,  9.0000, -0.5235,  ..., -0.4227, -0.3370, -0.5529]],
       grad_fn=<CatBackward0>)

In [14]:
# save my tensor data

torch.save(X_train,'train_inputs')
torch.save(train_target,'train_outputs')

torch.save(X_val,'val_inputs')
torch.save(val_target,'val_outputs')

torch.save(X_test,'test_inputs')
torch.save(test_target,'test_outputs')